In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

RAG Made Easy w/ Snowflake Cortex
========

Creating an end-to-end Retrieval Augmented Generation process (or RAG) directly in Snowflake.
1) Extract full text from PDF files using Cortex and PARSE_DOC.
2) Chunk those documents using SPLIT_TEXT.
3) Use Cortex Search build a custom search service for your store of knowledge.
4) Use foundational models to create a tailored answer.
5) Test with a mini Streamlit app

Libraries / packages used:
- ~~PyPDF2 : reading of PDF files~~ [Replaced with PARSE_DOCUMENT]
- ~~Langchain : for chunking~~ [Replaced with SPLIT_TEXT]
- Snowpark ML Python : Python API for Snowflake
- Snowflake Core : Rest API for Snowflake Service

In [ ]:
-- Optional set up: Place your MD files in a stage for extraction
ls @huberman;

In [ ]:
CREATE OR REPLACE TABLE NEW_RAW_TEXT AS
SELECT
    relative_path as episode_name
    , file_url
    , TO_VARCHAR(
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '@LLM_DEMO.PODCASTS.HUBERMAN',
        relative_path,
        {'mode': 'LAYOUT'}):content
    ) AS raw_text
from directory(@huberman);

In [ ]:
UPDATE
  NEW_RAW_TEXT
SET
  raw_text = REPLACE (raw_text, 'Transcribed by readthatpodcast.com', '');


SELECT * FROM NEW_RAW_TEXT LIMIT 1;

In [ ]:
--Optional : This no longer fails due to exceeding token limits, but chunking is still important!
SELECT
SNOWFLAKE.CORTEX.SUMMARIZE(raw_text)
FROM
NEW_RAW_TEXT
LIMIT 1;

In [ ]:
SELECT SNOWFLAKE.CORTEX.COUNT_TOKENS('summarize' , raw_text ) FROM
NEW_RAW_TEXT
LIMIT 1;

A note on chunking
-----
Chunking is the process of splitting a large body of text into smaller 'chunks' whilst attempting to keep as much relevant information as possible. Make the chunks too small and you run the risk of removing key information that the model requires to answer the question. Too large and it may be harder to retreive the correct body of text from the vector search - or spend tokens excessively.

There are many strategies towards chunking. Eg - pass the most relevant, top n relevant chunks, or pass the most relevent chunk + the chunk either side of that one. Play around and see what works for your use case!


In [ ]:
--Create the chunked version of the table
CREATE OR REPLACE TABLE NEW_CHUNK_TEXT AS
SELECT
        episode_name,
        TO_VARCHAR(c.value) as chunk
    FROM new_raw_text,
   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
      raw_text,
      'none',
      4000, --how many characters per row
      0 --how much overlap should there be?
   )) c;

In [ ]:
UPDATE
  NEW_CHUNK_TEXT
SET
  chunk = REPLACE (chunk, 'Transcribed by readthatpodcast.com', '');

In [ ]:
SELECT * FROM NEW_CHUNK_TEXT LIMIT 10;

In [ ]:
SELECT EPISODE_NAME, COUNT(*) FROM NEW_CHUNK_TEXT GROUP BY 1;

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE NEW_HUBERMAN
  ON CHUNK
  ATTRIBUTES EPISODE_NAME
  WAREHOUSE = tc_wh
  TARGET_LAG = '7 days'
  AS (
    (
    SELECT
        CHUNK,
        EPISODE_NAME
    FROM NEW_CHUNK_TEXT
)
);

In [ ]:
--query it with SQL
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'NEW_HUBERMAN',
      '{
         "query": "How do I make time go slower?",
         "columns":[
            "CHUNK",
            "EPISODE_NAME"
         ],
         "limit":3
      }'
  )
)['results'] as results;

In [ ]:
#query it with Python
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session
import streamlit as st
import json
import pandas as pd

session = get_active_session()

prompt="How do I make time go slower?"

root = Root(session)

# query service
svc = (root
  .databases["LLM_DEMO"]
  .schemas["PODCASTS"]
  .cortex_search_services["NEW_HUBERMAN"]
)

resp = svc.search(
  query=prompt,
  columns=["CHUNK", "EPISODE_NAME"],
  limit=3
).to_json()

#optional - I just like the way this looks...
json_conv = json.loads(resp) if isinstance(resp, str) else resp
search_df = pd.json_normalize(json_conv['results'])

#st.write(search_df)
for _, row in search_df.iterrows():
    st.write(f"**{row['EPISODE_NAME']}**")
    st.caption(row['CHUNK'])
    st.write('---')

In [ ]:
--Pass the chunk we need along with the prompt to get a better structured answer from the LLM  -- all in SQL!
SELECT snowflake.cortex.complete(
    'llama3.1-8b', 
    CONCAT( 
        'Answer the question based on the context. Context: ',
        (
            SELECT PARSE_JSON(
      SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
          'NEW_HUBERMAN',
          '{
             "query": "How do I make time go slower?",
             "columns":[
                "CHUNK",
                "EPISODE_NAME"
             ],
             "limit":3
              }'
          )
)['results'] as results
        ),
        'Question: ', 
        'How do I make time go slower?',
        'Answer: '
    )
) as response;

In [ ]:
from snowflake.cortex import Complete

st.title("Ask Your Data Anything :snowflake:")
st.write("""Built using end-to-end RAG in Snowflake with Cortex functions.""")

model = st.selectbox('Select your model:',('mistral-large2','mistral-7b','llama3.1-8b','llama3.1-70b'))

prompt = st.text_input("Enter prompt", placeholder="What makes time perceived to be slower?", label_visibility="collapsed") 

resp = svc.search(
  query=prompt,
  columns=["CHUNK", "EPISODE_NAME"],
  limit=7
).to_json()

system_p = f'''
Answer the question ONLY using the context provided. Here is the context to use: 
### Context: 
{resp}'
### Question:
{prompt}
'''

if prompt:
    LLM = Complete(model,system_p)
    st.write(LLM)